<a href="https://colab.research.google.com/github/joshba06/Object_Detection/blob/main/PreparationAndTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Readme

**Step 1**: 

This script has to be run on both, Colab (training) and the local machine (object detection). Set the variable "system_id" to 1, if running on Colab or to 0 if running on local machine

# 1. Getting Started

## 1.1 Define parameters

In [1]:
# Choose 1, if running on colab and 0 if running on local machine
system_id = 0

# Define the objects that you would like to train the deep learning-model with below
labels = ['Engine']

# Change model url and name if model changes
pre_trained_model_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz'
pre_trained_model_name = 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8'
custom_model_name = 'my_ssd_mobilenet_v2_fpnlite'

img_size = (640, 640)

## 1.2 Create file and folder structure

In [2]:
import os
import shutil

# Local machine
if system_id == 0:
    home_path = '/Users/niklas/Virtual_Environment/Version_5/projectAutonomous'
    print('Running on local machine...')

# Google colab    
elif system_id == 1:
    
    print('Running on Google Colab...')
    
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Setup path to home directory 
    home_path = '/content/drive/MyDrive/Colab_Notebooks/Object_Detection'
    os.chdir(home_path)    

else:
    print('No operating system was defined...')

Running on local machine...


In [3]:
import setup
files, paths = setup.createFolderStructure(labels, home_path, system_id)

Successfully created /Users/niklas/Virtual_Environment/Version_5/projectAutonomous/2_Tensorflow from scratch. 
Successfully created /Users/niklas/Virtual_Environment/Version_5/projectAutonomous/2_Tensorflow/protoc from scratch. 
Successfully created /Users/niklas/Virtual_Environment/Version_5/projectAutonomous/2_Tensorflow/workspace from scratch. 
Successfully created /Users/niklas/Virtual_Environment/Version_5/projectAutonomous/2_Tensorflow/workspace/scripts from scratch. 
Successfully created /Users/niklas/Virtual_Environment/Version_5/projectAutonomous/2_Tensorflow/workspace/training from scratch. 
Successfully created /Users/niklas/Virtual_Environment/Version_5/projectAutonomous/2_Tensorflow/workspace/training/annotations from scratch. 
Successfully created /Users/niklas/Virtual_Environment/Version_5/projectAutonomous/2_Tensorflow/workspace/training/images/training from scratch. 
Successfully created /Users/niklas/Virtual_Environment/Version_5/projectAutonomous/2_Tensorflow/workspa

In [4]:
# Create dict with labels
dict_labels = {}
num = 1
for label in labels:
    dict_labels[label]= num
    num += 1
print(dict_labels)

{'Engine': 1}


## 1.3 Installing Dependencies and importing modules

In [5]:
# Local machine
if system_id == 0:
    try:
        import cv2 as cv
        if cv.__version__ == "4.6.0":
            print('OpenCV version 4.6.0 is already installed...')
        else:
            !pip uninstall opencv-python -y
            !pip install opencv-python==4.6.0
    except:
            !pip uninstall opencv-python -y
            !pip install opencv-python==4.6.0

OpenCV version 4.6.0 is already installed...


In [6]:
!pip install wget

In [7]:
import cv2 as cv

import uuid

import time

import pathlib

import shutil

import math

import wget

# 3. Prepare Tensorflow Object Detection API

## 3.1 Install dependencies

In [8]:
# Local machine
if system_id == 0:
    try:
        import tensorflow as tf
        if tf.__version__ == "2.5.0":
            print('Tenforflow version 2.5.0 is already installed...')
        else:
            print('Tensorflow version is: '+str(tf.__version__))
            !pip install tensorflow==2.5.0
    except:
        !pip install tensorflow==2.5.0

# Google colab    
elif system_id == 1:
    !pip install tensorflow-gpu==2.5.0 

else:
    print('No operating system was defined...')

Tenforflow version 2.5.0 is already installed...


In [9]:
# Download the model garden (model garden is an environment that is necessary to train new models from scratch or to continue training existing models)
# The model itself will be downloaded later

# Clone repository only if it does not exist already
os.chdir(paths['2_Tensorflow'])
if os.path.exists(paths['2_Tensorflow']+'/models/research') is False:
    print('Cloning model garden..')
    !git clone https://github.com/tensorflow/models.git
    
else:
    print('Model garden is already installed...')
    
os.chdir(paths['home'])

Cloning model garden..
Cloning into 'models'...
remote: Enumerating objects: 76179, done.
remote: Counting objects: 100% (641/641), done.
remote: Compressing objects: 100% (321/321), done.
remote: Total 76179 (delta 365), reused 553 (delta 317), pack-reused 75538
Receiving objects: 100% (76179/76179), 592.74 MiB | 473.00 KiB/s, done.
Resolving deltas: 100% (53976/53976), done.
Updating files: 100% (3125/3125), done.


In [10]:
# Install protobuf
if os.path.exists(paths['2_Tensorflow']+'/protoc/protoc-21.1-osx-aarch_64.zip') is False:

    # Go to destination directory
    os.chdir(paths['protoc'])
    protoc_url = 'https://github.com/protocolbuffers/protobuf/releases/download/v21.1/protoc-21.1-osx-aarch_64.zip'
    wget.download(protoc_url)

    # Extract all content of downloaded file
    from zipfile import ZipFile

    with ZipFile('protoc-21.1-osx-aarch_64.zip', 'r') as zipObj:
        zipObj.extractall()

    os.environ['Path'] = paths['protoc']+'/bin'
    os.chdir(paths['research'])

    !protoc object_detection/protos/*.proto --python_out=.

else:
    print('Protobuf is already installed...')
    
os.chdir(paths['home'])

100% [..........................................................................] 1357797 / 1357797

In [11]:
# Install pycocotools

# Clone repository only if it does not exist already
if os.path.exists(paths['research']+'/cocoapi') is False:
    print('Cloning cocoapi..')
    !git clone https://github.com/cocodataset/cocoapi.git
    
    # Moving cloned file to 'research' folder
    destination = paths['research']
    source = paths['home']+'/cocoapi'
    shutil.move(source, destination)
    
else:
    print('Cocoapi is already installed...')

os.chdir(paths['home'])

Cloning cocoapi..
Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 10.70 MiB/s, done.
Resolving deltas: 100% (576/576), done.


## 3.2 Install Tensorflow Object Detection API

In [12]:
# Check if API has already been installed
if os.path.exists(paths['research']+'/setup.py') is False:
    print('Installing setup.py...')
    
    # Move to 'research' directory
    os.chdir(paths['research'])

    # Copy setup.py to current working directory
    !cp object_detection/packages/tf2/setup.py .

    # Execute setup.py (this command installs all dependencies needed for tf2 odapi)
    !python -m pip install .

    print('Installation complete..')

else:
    print('Object Detection API is already installed...')

# Move back to home-directory
os.chdir(paths['home'])

Installing setup.py...
Processing /Users/niklas/Virtual_Environment/Version_5/projectAutonomous/2_Tensorflow/models/research
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 11.1 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached opencv_python_headless-4.6.0.66-cp36-abi3-macosx_10_15_x86_64.whl (46.4 MB)
  Using cached tensorflow-2.9.1-cp39-cp39-macosx_10_14_x86_64.whl (228.5 MB)


  Using cached numpy-1.22.4-cp39-cp39-macosx_10_15_x86_64.whl (17.7 MB)
  Using cached opencv_python-4.6.0.66-cp36-abi3-macosx_10_15_x86_64.whl (46.4 MB)
  Using cached tensorflow_estimator-2.9.0-py2.py3-none-any.whl (438 kB)
  Using cached absl_py-1.2.0-py3-none-any.whl (123 kB)


  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1661667 sha256=c56802a369f5e7fec10fe170dbfe7f42c3227a832a3160e5a847b6c87aacb721
  Stored in directory: /private/var/folders/hd/gz2v7mz153nbydn703wyfm140000gn/T/pip-ephem-wheel-cache-z99ezri3/wheels/83/96/06/bf1c56d4991d748b69d9bedc3277c1b0f0d40a09a781e4d7c8
Successfully built object-detection
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.15.0
    Uninstalling absl-py-0.15.0:
      Successfully uninstalled absl-py-0.15.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.5.0
    Uninstall

## 3.3 Check if API was installed successfully

In [13]:
# Move to 'research' directory
os.chdir(paths['research'])
import object_detection

# Local machine
if system_id == 0:
    !python {paths['research']+'/object_detection/builders/model_builder_tf2_test.py'}
    

# Google colab    
elif system_id == 1:
    !pip install numpy --upgrade # This had to be added for execution on colab. Problem solved using stackoverflow

    !python {paths['research']+'/object_detection/builders/model_builder_tf2_test.py'}
else:
    print('No operating system was defined...')

# Move back to home directory
os.chdir(paths['home'])

Running tests under Python 3.9.6: /Users/niklas/Virtual_Environment/Version_5/joshBakNew/bin/python
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-08-15 09:17:41.474209: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/niklas/Virtual_Environment/Version_5/joshBakNew/lib/python3.9/site-packages/object_detection/builders/model_builder.py:1102: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn(('Building experimental DeepMAC meta-arch.'
W0815 09:17:41.844075 4641814016 model_builder.py:1102] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 0.71s
I0815 09:1

I0815 09:17:47.158477 4641814016 efficientnet_model.py:143] round_filter input=40 output=40
I0815 09:17:47.158608 4641814016 efficientnet_model.py:143] round_filter input=80 output=80
I0815 09:17:47.440899 4641814016 efficientnet_model.py:143] round_filter input=80 output=80
I0815 09:17:47.441029 4641814016 efficientnet_model.py:143] round_filter input=112 output=112
I0815 09:17:47.690235 4641814016 efficientnet_model.py:143] round_filter input=112 output=112
I0815 09:17:47.690370 4641814016 efficientnet_model.py:143] round_filter input=192 output=192
I0815 09:17:48.061171 4641814016 efficientnet_model.py:143] round_filter input=192 output=192
I0815 09:17:48.061316 4641814016 efficientnet_model.py:143] round_filter input=320 output=320
I0815 09:17:48.156335 4641814016 efficientnet_model.py:143] round_filter input=1280 output=1280
I0815 09:17:48.204802 4641814016 efficientnet_model.py:453] Building model efficientnet with params ModelConfig(width_coefficient=1.0, depth_coefficient=1.0, 

I0815 09:17:52.836718 4641814016 efficientnet_model.py:453] Building model efficientnet with params ModelConfig(width_coefficient=1.1, depth_coefficient=1.2, resolution=260, dropout_rate=0.3, blocks=(BlockConfig(input_filters=32, output_filters=16, kernel_size=3, num_repeat=1, expand_ratio=1, strides=(1, 1), se_ratio=0.25, id_skip=True, fused_conv=False, conv_type='depthwise'), BlockConfig(input_filters=16, output_filters=24, kernel_size=3, num_repeat=2, expand_ratio=6, strides=(2, 2), se_ratio=0.25, id_skip=True, fused_conv=False, conv_type='depthwise'), BlockConfig(input_filters=24, output_filters=40, kernel_size=5, num_repeat=2, expand_ratio=6, strides=(2, 2), se_ratio=0.25, id_skip=True, fused_conv=False, conv_type='depthwise'), BlockConfig(input_filters=40, output_filters=80, kernel_size=3, num_repeat=3, expand_ratio=6, strides=(2, 2), se_ratio=0.25, id_skip=True, fused_conv=False, conv_type='depthwise'), BlockConfig(input_filters=80, output_filters=112, kernel_size=5, num_repeat=

I0815 09:17:59.467200 4641814016 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet EfficientNet backbone version: efficientnet-b5
I0815 09:17:59.467331 4641814016 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num filters: 288
I0815 09:17:59.467387 4641814016 ssd_efficientnet_bifpn_feature_extractor.py:148] EfficientDet BiFPN num iterations: 7
I0815 09:17:59.469196 4641814016 efficientnet_model.py:143] round_filter input=32 output=48
I0815 09:17:59.489827 4641814016 efficientnet_model.py:143] round_filter input=32 output=48
I0815 09:17:59.489955 4641814016 efficientnet_model.py:143] round_filter input=16 output=24
I0815 09:17:59.679039 4641814016 efficientnet_model.py:143] round_filter input=16 output=24
I0815 09:17:59.679171 4641814016 efficientnet_model.py:143] round_filter input=24 output=40
I0815 09:18:00.104116 4641814016 efficientnet_model.py:143] round_filter input=24 output=40
I0815 09:18:00.104250 4641814016 efficientnet_model.py:143] round_fil

I0815 09:18:08.931744 4641814016 efficientnet_model.py:143] round_filter input=24 output=48
I0815 09:18:08.931930 4641814016 efficientnet_model.py:143] round_filter input=40 output=80
I0815 09:18:09.459733 4641814016 efficientnet_model.py:143] round_filter input=40 output=80
I0815 09:18:09.459865 4641814016 efficientnet_model.py:143] round_filter input=80 output=160
I0815 09:18:10.414225 4641814016 efficientnet_model.py:143] round_filter input=80 output=160
I0815 09:18:10.414355 4641814016 efficientnet_model.py:143] round_filter input=112 output=224
I0815 09:18:11.511564 4641814016 efficientnet_model.py:143] round_filter input=112 output=224
I0815 09:18:11.511702 4641814016 efficientnet_model.py:143] round_filter input=192 output=384
I0815 09:18:13.238204 4641814016 efficientnet_model.py:143] round_filter input=192 output=384
I0815 09:18:13.238332 4641814016 efficientnet_model.py:143] round_filter input=320 output=640
I0815 09:18:13.864125 4641814016 efficientnet_model.py:143] round_fi

# 4. Prepare new training job

## 4.1 Install dependencies and import modules (only on Colab)

In [14]:
# Local machine
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:
  
  ## Install missing modules for randomTrafficSign
  !pip install matplotlib
  !pip install lxml

  import numpy as np
  from xml.etree.ElementTree import ElementTree
  from xml.etree.ElementTree import Element
  import xml.etree.ElementTree as etree
  import xml.dom.minidom

  from lxml import etree
  os.chdir(paths['home'])

else:
    print('No operating system was defined...')

Skipping step


## 4.2 Partition images for testing and training (only on Colab)

**Important**: Images of objects must be in the following format: "Mug_1.jpg", "Cat_3.jpg" and must be located in their respective folders

In [15]:
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:
    !pip install -U albumentations --no-binary qudida,albumentations

Skipping step


In [16]:
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:

    # imagePreprocessing.py must be in same directory as this main script
    %matplotlib inline
    path = paths['0_User_Input']+'/scripts'
    os.chdir(path)
    import imageProcessing as imgprep
    os.chdir(paths['home'])

    imgprep.clearPreprocessing(paths['images'])
    imgprep.clearTraining(paths['training']+'/images')

    numImgs = 150
    upperScale = 150
    lowerScale = 80

    # Modify and multiply images and store in 1_Preprocessing folder
    imgprep.main(dict_labels, paths['backgrounds'], paths['objects'], numImgs, upperScale, lowerScale, paths['images']+'/', paths['training']+'/')

    imgprep.clearPreprocessing(paths['images'])

Skipping step


## 5.4 Create labelmap (only on Colab)

In [17]:
# Local machine
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:

    # Convert label-dict to needed format
    labelmap = []
    for key in dict_labels:
      temp = {}
      temp['name'] = key
      temp['id'] = dict_labels[key]
      labelmap.append(temp)
    print(labelmap)

    files['labelmap'] = paths['annotations']+'/label_map.pbtxt'

    with open(files['labelmap'], 'w') as file:
      for label in labelmap:
          file.write('item { \n')
          file.write('\tname:\'{}\'\n'.format(label['name']))
          file.write('\tid:{}\n'.format(label['id']))
          file.write('}\n')
        
else:
    print('No operating system was defined...')

Skipping step


In [18]:
# Install pandas
!pip install pandas

## 5.5 Create TFRecord (only on colab)

In [19]:
# Local machine
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:

  # Add labelmap and tfrecords to 'files' dictionary
  files['tf_train'] = paths['annotations']+'/train.record'
  files['tf_test'] = paths['annotations']+'/test.record'

  # Add line to download TF record file from nicknochnack

  # Copy generatetfrecord.py to scripts
  source = paths['0_User_Input']+'/scripts/generatetfrecord.py'
  shutil.copy(source, paths['scripts'])

  # Change directory to 'scripts'
  os.chdir(paths['workspace']+'/scripts')

  # Create / overwrite TFRecord files for training and testing

  # Create train data:
  !python generatetfrecord.py -x {paths['images_training']} -l {files['labelmap']} -o {files['tf_train']}

  # Create test data:
  !python generatetfrecord.py -x {paths['images_testing']} -l {files['labelmap']} -o {files['tf_test']}

  # Go back to home directory
  os.chdir(paths['home'])

 
else:
    print('No operating system was defined...')

Skipping step


## 5.6 Download pre-trained model (only on colab)



In [20]:
# Local machine
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:

  # Update the settings for the image import and multiplication script depending on which size of image the model uses!

  # Check if the chosen model has already been downloaded
  if os.path.exists(paths['pre_trained_models']+'/'+str(pre_trained_model_name)) is False:

      # Go to destination directory
      os.chdir(paths['pre_trained_models'])
      wget.download(pre_trained_model_url)

      # Extract all content of downloaded file
      import tarfile

      file = tarfile.open('ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz')

      file.extractall(paths['pre_trained_models'])

      file.close()
                    
      # Delete downloaded tar.gz file to save storage space
      # Add code here
      #
      #
      
      # Create new folder for this model in training/models
      paths['active_model'] = paths['models']+'/'+custom_model_name
      os.makedirs(paths['active_model'])
      
      print('Model was successfully downloaded...')


  else:
    print(str(pre_trained_model_name)+' was already installed...')
    
    os.chdir(paths['home'])

else:
    print('No operating system was defined...')

Skipping step


## 5.7 Update the config file and pipeline for the new training job

In [21]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

AttributeError: module 'tensorflow.python.util.dispatch' has no attribute 'add_fallback_dispatch_list'

In [ ]:
# Local machine
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:

  ## Copy or replace pipeline in active model directory
  files['pipeline_downloaded'] = paths['pre_trained_models']+'/'+pre_trained_model_name+'/pipeline.config'
  paths['active_model'] = paths['models']+'/'+custom_model_name
  files['pipeline_active'] = paths['active_model']+'/pipeline.config'
  paths['downloaded_model'] = paths['pre_trained_models']+'/'+pre_trained_model_name

  # If pipeline already exists in active directory, replace it
  if os.path.exists(files['pipeline_active']) == True:
      os.remove(files['pipeline_active'])
      shutil.copy(files['pipeline_downloaded'], paths['active_model'])
      print('Pipeline replaced in active model directory...')

  # If pipeline does not yet exist in active directory, copy it from downloaded model
  else:
      files['pipeline_downloaded'] = paths['pre_trained_models']+'/'+pre_trained_model_name+'/pipeline.config' 
      shutil.copy(files['pipeline_downloaded'], paths['active_model'])
      print('Pipeline copied to active model directory...')

  ## Configure pipeline

  config = config_util.get_configs_from_pipeline_file(files['pipeline_active'])
  pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
  with tf.io.gfile.GFile(files['pipeline_active'], "r") as f:                                                                                                                                                                                                                     
      proto_str = f.read()                                                                                                                                                                                                                                          
      text_format.Merge(proto_str, pipeline_config)  


  pipeline_config.model.ssd.num_classes = len(labels) # Number of labels the model should be trained for
  pipeline_config.train_config.batch_size = 4 # This should be the number of training jobs that run parallel

  # Get checkpoint 0 from (original) downloaded model 
  files['checkpoint0'] = paths['downloaded_model']+'/checkpoint/ckpt-0'

  pipeline_config.train_config.fine_tune_checkpoint = files['checkpoint0']

  pipeline_config.train_config.fine_tune_checkpoint_type = "detection"

  # Get labelmap
  pipeline_config.train_input_reader.label_map_path= files['labelmap']

  # Get TF-Record
  pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [files['tf_train']]
  pipeline_config.eval_input_reader[0].label_map_path = files['labelmap']
  pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [files['tf_test']]

  config_text = text_format.MessageToString(pipeline_config)

  # Update active pipeline
  with tf.io.gfile.GFile(files['pipeline_active'], "wb") as f:                                                                                                                                                                                                                     
      f.write(config_text)   
      
  print('Pipeline successfully configured...')  

  # Copy model_main_tf2.py to workspace -> training   'TensorFlow/models/research/' file to 
  source = paths['research']+'/object_detection/model_main_tf2.py'
  destination = paths['training']
  shutil.copy(source, destination)

else:
    print('No operating system was defined...')


# 6. Start new training job (only on colab)

In [ ]:
# Local machine
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:
  
  files['training_script'] = paths['training']+'/model_main_tf2.py'
  model_dir = paths['active_model']
  
  command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=10000".format(files['training_script'], model_dir, files['pipeline_active'])
  # first argument: Path to the model_main_tf2.py file
  # second argument: Path to the diretory in which the pipeline.config file is placed (not the path to the file itself)
  # third argument: Path to actual pipeline.config in active directory

  # This command is necessary to fix issue with training on colab
  # source: https://stackoverflow.com/questions/70998639/dnn-library-is-not-found-ssd-mobile-net-v2-in-colab#answer-72404540
  !apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

  !{command}

else:
  print('No operating system was defined...')

# 7 Evaluate training (both)

In [25]:
# Local machine
if system_id == 0:

  paths['train'] = paths['3_Output']+'/my_ssd_mobilenet_v2_fpnlite/train'
  paths['eval'] = paths['3_Output']+'/my_ssd_mobilenet_v2_fpnlite/eval'

  os.chdir(paths['eval'])
  !tensorboard --logdir=.

# Google colab    
elif system_id == 1:

  command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(files['training_script'], model_dir, files['pipeline_active'], model_dir)
  !{command}

else:
  print('No operating system was defined...')


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.9.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


# 8 Download model (Colab)

In [ ]:
# Local machine
if system_id == 0:
  print('Skipping step...')

# Google colab    
elif system_id == 1:

    # Copy labelmap to model folder
    source = files['labelmap']
    shutil.copy(source, paths['actt copy_tree
    copy_tree(paths['active_model'], paths['3_Output']+'/'+custom_model_name)

else:
  print('No operating system was defined...')